# GreenRivers MORO

_Description_

## Set up MORO

|Outcome of interest| Threshold  |
|-------------------|------------|
| Deaths            | $\leq$ 0.x|
| Damage            | $\leq$ 0.x |
| Dike Costs        | $\leq$ 0.x|   
| RfR Costs         | $\geq$ 0.x|

In [ ]:
from model.dike_model_function import DikeNetwork  # @UnresolvedImport
from model.problem_formulation import get_model_for_problem_formulation

In [ ]:
dike_model, planning_steps = get_model_for_problem_formulation(5)

In [3]:
# Define robustness functions
# We want a function that returns 0 for the outcome to be in the range that we want and higher otherwise.



In [ ]:
MAXIMIZE = ScalarOutcome.MAXIMIZE
MINIMIZE = ScalarOutcome.MINIMIZE

# These functions need to only return one value...

robustness_functions = [
    ScalarOutcome('Damage Score', function = MY_FUNC, kind = MINIMIZE,
                  variable_name=['A.1_Expected Annual Damage' ,'A.2_Expected Annual Damage',
                                 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage']),    
    ScalarOutcome('Deaths Score', function = MY_FUNC, kind = MINIMIZE
                  variable_name=['A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths',
                                 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths']),
    ScalarOutcome('Dike Invest Score', function = MY_FUNC, kind = MINIMIZE,
                  variable_name=['A.1_Dike Investment Costs', 'A.2_Dike Investment Costs',
                                 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs']),
    ScalarOutcome('RfR Invest Score', variable_name=['RfR Total Costs'], function = MY_FUNC, kind = MINIMIZE),
    ScalarOutcome('Evac Score', variable_name=['Expected Evacuation Costs'], function = MY_FUNC, kind = MINIMIZE),
]

In [ ]:
from dps_lake_model import lake_model
from ema_workbench import (Model, RealParameter, ScalarOutcome, ema_logging)
ema_logging.log_to_stderr(ema_logging.INFO)

lake_model = Model('lakeproblem', function = lake_model)

# Define uncertainties
lake_model.uncertainties = [RealParameter('mean', 0.01, 0.05),
                            RealParameter('stdev', 0.001, 0.005),
                            RealParameter('b', 0.1, 0.45),
                            RealParameter('q', 2, 4.5),
                            RealParameter('delta', 0.93, 0.99)]

# Define lever
lake_model.levers = [RealParameter('c1', -2, 2),
                        RealParameter('c2', -2, 2),
                        RealParameter('r1', 0, 2),
                        RealParameter('r2', 0, 2),
                        RealParameter('w1', 0, 1)]

# Define outcomes
lake_model.outcomes = [ScalarOutcome('max_P', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('utility', kind=ScalarOutcome.MAXIMIZE),
                       ScalarOutcome('inertia', kind=ScalarOutcome.MAXIMIZE),
                       ScalarOutcome('reliability', kind=ScalarOutcome.MAXIMIZE)]

In [ ]:
from ema_workbench.em_framework import sample_uncertainties

n_scenarios = 2
scenarios = sample_uncertainties(lake_model, n_scenarios)
nfe = int(2)

In [ ]:
from ema_workbench import ema_logging
from ema_workbench.em_framework.optimization import (HyperVolume, 
                                                     EpsilonProgress)
from ema_workbench.em_framework.evaluators import BaseEvaluator
import time

BaseEvaluator.reporting_frequency = 0.1
ema_logging.log_to_stderr(ema_logging.INFO)

epsilons = [0.05,]*len(robustness_functions)
convergence = [HyperVolume(minimum=[0,0,0], maximum=[1.1, 1.1, 1.1]),
              EpsilonProgress()]

start = time.time()

with MultiprocessingEvaluator(lake_model) as evaluator:
    results, convergence = evaluator.robust_optimize(robustness_functions,
                                                    scenarios = scenarios,
                                                    nfe=nfe,
                                                    epsilons=epsilons,
                                                    convergence=convergence)
end = time.time()
print("Time taken: {:0.5f} minutes".format((end - start)/60))

## Evaluate $\epsilon$-convergence

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, figsize=(8,4))
ax1.plot(convergence.nfe, convergence.epsilon_progress)
ax1.set_ylabel('$\epsilon$-progress')
ax2.plot(convergence.nfe, convergence.hypervolume)
ax2.set_ylabel('hypervolume')

ax1.set_xlabel('number of function evaluations')
ax2.set_xlabel('number of function evaluations')
plt.show()

## Re-evaluate under more scenarios

In [ ]:
# policies = ??

In [ ]:
start = time.time()
with MultiprocessingEvaluator(lake_model) as evaluator:
    results = evaluator.perform_experiments(scenarios = 1000, policies = policies)
end = time.time()
print("Time taken: {:0.5f} minutes".format((end - start)/60))

We can also evaluate regret compared to a base case.

In [ ]:
def calculate_regret(data, best):
    return np.abs(best-data)

In [ ]:
experiments, outcomes = results

overall_regret = {}
max_regret = {}
for outcome in model.outcomes:
    policy_column = experiments['policy']
    
    # create a DataFrame with all the relevent information
    # i.e., policy, scenario_id, and scores
    data = pd.DataFrame({outcome.name: outcomes[outcome.name], 
                         "policy":experiments['policy'],
                         "scenario":experiments['scenario']})
    
    # reorient the data by indexing with policy and scenario id
    data = data.pivot(index='scenario', columns='policy')
    
    # flatten the resulting hierarchical index resulting from 
    # pivoting, (might be a nicer solution possible)
    data.columns = data.columns.get_level_values(1)
    
    # we need to control the broadcasting. 
    # max returns a 1d vector across scenario id. By passing
    # np.newaxis we ensure that the shape is the same as the data
    # next we take the absolute value
    #
    # basically we take the difference of the maximum across 
    # the row and the actual values in the row
    #
    outcome_regret = (data.max(axis=1)[:, np.newaxis] - data).abs()
    
    overall_regret[outcome.name] = outcome_regret
    max_regret[outcome.name] = outcome_regret.max()
    

In [ ]:
max_regret = pd.DataFrame(max_regret)
sns.heatmap(max_regret/max_regret.max(), cmap='viridis', annot=True)
plt.show()

In [ ]:
colors = sns.color_palette()

data = max_regret

# makes it easier to identify the policy associated with each line
# in the parcoords plot
# data['policy'] = data.index.astype("float64")

limits = parcoords.get_limits(data)
limits.loc[0, ['utility', 'inertia', 'reliability', 'max_P']] = 0

paraxes = parcoords.ParallelAxes(limits)
for i, (index, row) in enumerate(data.iterrows()):
    paraxes.plot(row.to_frame().T, label=str(index), color=colors[i])
paraxes.legend()
    
plt.show()

We see striking differences between blue and orange (1 and 3) and green and red (5 and 8). The first two options have low regret on the first three objectives, but higher regret on utility. For the second two options it is reversed

Note that we have been looking at the maximum regret. I also saved the distribution of regret over the set of scenarios. So let's visualize this and see what we can learn from it

In [ ]:
from collections import defaultdict

policy_regret = defaultdict(dict)
for key, value in overall_regret.items():
    for policy in value:
        policy_regret[policy][key] = value[policy]

In [ ]:
# this generates a 2 by 2 axes grid, with a shared X and Y axis
# accross all plots
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(10,10), 
                         sharey=True, sharex=True)

# to ensure easy iteration over the axes grid, we turn it
# into a list. Because there are four plots, I hard coded
# this. 
axes = [axes[0,0], axes[0,1],
        axes[1,0],]

# zip allows us to zip together the list of axes and the list of 
# key value pairs return by items. If we iterate over this
# it returns a tuple of length 2. The first item is the ax
# the second items is the key value pair.
for ax, (policy, regret) in zip(axes, policy_regret.items()):
    data = pd.DataFrame(regret)

    # we need to scale the regret to ensure fair visual
    # comparison. We can do that by divding by the maximum regret
    data = data/max_regret.max(axis=0)
    sns.boxplot(data=data, ax=ax)
    
    # removes top and left hand black outline of axes
    sns.despine()
    
    # ensure we know which policy the figure is for
    ax.set_title(str(policy))
plt.show()